In [130]:
#!python -m pip install pyyaml==5.4.1 -q
#!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13' -q

In [131]:
import json
import pandas as pd
import numpy as np
import shutil
import os
import random

In [132]:
data_dir = "../Dataset/GlobalWheatDetection"

In [133]:
os.system(f"mkdir {data_dir}/images")
os.system(f"mkdir {data_dir}/converted")
os.system(f"mkdir {data_dir}/converted/train")
os.system(f"mkdir {data_dir}/converted/test")

0

In [134]:
data_dir

'../Dataset/GlobalWheatDetection'

In [135]:
df = pd.read_csv(f"{data_dir}/train.csv")

In [136]:
df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [137]:
im_idx = 0
ann_idx = 0
annotations = []
images = []
categories = [
    {'id':0,'name':'wheat'}
]

images_dir = f"{data_dir}/train"
imgs = df.image_id.value_counts().index.tolist()

for im in imgs:
    
    anns = df[df.image_id==im]
    
    img_obj = {
        'id': im_idx,
        'file_name': f"{im}.jpg",
        'height': int(anns.iloc[0]['height']),
        'width': int(anns.iloc[0]['width'])
    }

    images.append(img_obj)
    os.system(f"cp {images_dir}/{im}.jpg {data_dir}/images/{im}.jpg")

    for ann in anns['bbox']:
        
        bbox = [float(a) for a in ann.replace("[","").replace(']','').split(',')]
        ann_obj = {
            'id': ann_idx,
            'image_id': im_idx,
            'category_id': 0,
            'area': int(bbox[2] * bbox[3]),
            'bbox': bbox,
            'iscrowd':0,
        }

        annotations.append(ann_obj)

        ann_idx += 1
    im_idx += 1

In [138]:
test_images = random.sample(images, 50)
test_annotations = []
for im in test_images:
    anns = [ann for ann in annotations if ann['image_id'] == im['id']]
    test_annotations.extend(anns)
    for ann in anns:
        annotations.remove(ann)
    images.remove(im)
    
    os.system(f"mv {data_dir}/images/{im['file_name']} {data_dir}/converted/test/{im['file_name']}")

In [139]:
os.system(f'mv {data_dir}/images/* {data_dir}/converted/train')

0

In [140]:
train = {
    'images':images,
    'annotations':annotations,
    'categories': categories
}

test = {
    'images':test_images,
    'annotations':test_annotations,
    'categories': categories
}

with open(f"{data_dir}/converted/train/train.json", "w") as f:
    json.dump(train, f)
    
with open(f"{data_dir}/converted/test/test.json", "w") as f:
    json.dump(test, f)